# Lab 6 - Dependency parsing using machine learning techniques
Laboration 6 in EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/assignment-6/

Author: Jonatan Kronander

## Objectives
The objectives of this assignment are to:
* Extract feature vectors and train a classifier
* Write a statistical dependency parser
* Understand how to design parameter sets
* Write a short report on your results

## Training the classifiers
If you have not done it in the previous assignment, for each data set you have generated, fit a corresponding model using logistic regression (or another classifier if you want) and save it.

In [2]:
import pickle

with open('X_feature1.pickle', 'rb') as handle:
    X1 = pickle.load(handle)

with open('Y_feature1.pickle', 'rb') as handle:
    Y1 = pickle.load(handle)

with open('X_feature2.pickle', 'rb') as handle:
    X2 = pickle.load(handle)
    
with open('Y_feature2.pickle', 'rb') as handle:
    Y2 = pickle.load(handle)
    
with open('X_feature3.pickle', 'rb') as handle:
    X3 = pickle.load(handle)
    
with open('Y_feature2.pickle', 'rb') as handle:
    Y3 = pickle.load(handle)